In [1]:
!pip install pyLDAvis --quiet
!pip install chart_studio --quiet


     |████████████████████████████████| 1.6MB 5.8MB/s 
     |████████████████████████████████| 71kB 3.3MB/s 


In [2]:
import pandas as pd
import numpy as np
import time
import re
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import LatentDirichletAllocation
import gensim
from spacy.tokenizer import Tokenizer
import gensim.corpora as corpora
from gensim.models.ldamulticore import LdaMulticore
from pprint import pprint
from gensim.models.coherencemodel import CoherenceModel
import plotly.express as px
import pyLDAvis.gensim
import chart_studio
import chart_studio.plotly as py 
import chart_studio.tools as tls
from operator import itemgetter
from ipywidgets import interact
import tqdm
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets


# supress warnings
warnings.filterwarnings("ignore")

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


We will load in our preprocessed tweets from our [data cleaning notebook](https://github.com/tarrantcarter/Final_Capstone/blob/main/Modern_Motivation_Data_Cleaning_Feature_Engineering.ipynb). The csv can be found [here](https://drive.google.com/file/d/1-d-61YuocweY0F3rSnf-idL1ixbuKq4H/view?usp=sharing). 

In [4]:
# load in cleaned tweets from data cleaning notebook
tweets_cleaned = pd.read_json("/content/drive/MyDrive/Data/NLP_Capstone/motivational_tweets_cleaned.json")

In [5]:
tweets_cleaned.head()

,date,user_name,content,content_preprocessed,unigram_tokens,ngrams,ngram_tokens,nouns_only,nouns_verbs,bigrams_trigrams,bigrams_trigrams_strings
44,2021-01-17 22:13:17,LewisHowes,Know this. Everything is happening for a reaso...,know happen reason favor betterment future pai...,"[know, happen, reason, favor, betterment, futu...",know_happen_reason favor betterment future pai...,"[know_happen_reason, favor, betterment, future...","[reason, favor, betterment, future, pain, feel...","[know, reason, favor, betterment, future, pain...",[know_happen_reason],know_happen_reason
61,2021-01-15 15:28:06,LewisHowes,Protect your inner peace at all costs. Create ...,protect inner peace cost create daily practice...,"[protect, inner, peace, cost, create, daily, p...",protect inner_peace cost create daily_practice...,"[protect, inner_peace, cost, create, daily_pra...","[peace, cost, practice, communicate, stress, d...","[peace, cost, practice, communicate, stress, d...","[inner_peace, daily_practice]",inner_peace daily_practice
161,2021-01-07 16:00:29,LewisHowes,Always remember to ask for exactly what you wa...,remember ask exactly want ask love good health...,"[remember, ask, exactly, want, ask, love, good...",remember ask exactly want ask love good health...,"[remember, ask, exactly, want, ask, love, good...","[health, abundance, peace, ask, wisdom, creati...","[remember, health, abundance, peace, ask, wisd...","[health_abundance, ask_wisdom]",health_abundance ask_wisdom
274,2021-01-01 02:44:46,LewisHowes,This will be your greatest year ever. All your...,great year work start pay earn happy love deep...,"[great, year, work, start, pay, earn, happy, l...",great year work start pay earn happy love deep...,"[great, year, work, start, pay, earn, happy, l...","[year, work, start, earn, love, embrace, fear,...","[year, work, start, pay, earn, love, embrace, ...",[massive_action],massive_action
317,2020-12-30 01:38:35,LewisHowes,Be grateful for the breakdown this year. It's ...,grateful breakdown year set massive breakthrou...,"[grateful, breakdown, year, set, massive, brea...",grateful breakdown year set massive breakthrou...,"[grateful, breakdown, year, set, massive, brea...","[year, breakthrough, money, mission, get, rela...","[year, set, breakthrough, money, come, mission...",[real_friend],real_friend


In [6]:
tweets_cleaned.applymap(type)

,date,user_name,content,content_preprocessed,unigram_tokens,ngrams,ngram_tokens,nouns_only,nouns_verbs,bigrams_trigrams,bigrams_trigrams_strings
44,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
61,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
161,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
274,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
317,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
...,...,...,...,...,...,...,...,...,...,...,...
977598,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
977599,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
977606,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
977608,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>


In [7]:
tweets_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 621449 entries, 44 to 977610
Data columns (total 11 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   date                      621449 non-null  datetime64[ns]
 1   user_name                 621449 non-null  object        
 2   content                   621449 non-null  object        
 3   content_preprocessed      621449 non-null  object        
 4   unigram_tokens            621449 non-null  object        
 5   ngrams                    621449 non-null  object        
 6   ngram_tokens              621449 non-null  object        
 7   nouns_only                621449 non-null  object        
 8   nouns_verbs               621449 non-null  object        
 9   bigrams_trigrams          621449 non-null  object        
 10  bigrams_trigrams_strings  621449 non-null  object        
dtypes: datetime64[ns](1), object(10)
memory usage: 56.9+ MB


In [11]:
tweets_cleaned.shape

(621449, 11)

# Topic Modeling

In [12]:
# create dictionary
id2word = corpora.Dictionary(tweets_cleaned['unigram_tokens'])
# create texts corpus
texts = tweets_cleaned['unigram_tokens']
# term document frequency
corpus = [id2word.doc2bow(text) for text in texts]
# print first 30 tuples from corpus
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1)]


In [13]:
# number of topics
num_topics = 10
# build LDA model
base_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# # print the keyword in the 10 topics
# pprint(base_model.print_topics())
# doc_lda = base_model[corpus]

In [14]:
# filtering for words 
words = [re.findall(r'"([^"]*)"',t[1]) for t in base_model.print_topics()]

# create topic sorted by 10 most relevent words
topics = [' '.join(t[0:10]) for t in words]


# print most relevent words for each topic
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
look good think man peace great thing strength place ralph

------ Topic 1 ------
change thing love life choose peale hard work time worth

------ Topic 2 ------
u like self win reality allow churchill winston soul have

------ Topic 3 ------
life get proverb better person happiness find important expect practice

------ Topic 4 ------
dyer life live day love light fear time thought way

------ Topic 5 ------
man time amp choice lincoln abraham power happen matter good

------ Topic 6 ------
know amp people need way think ask world life well

------ Topic 7 ------
feel want good heart mean thing close lose keep kill

------ Topic 8 ------
mind today come moment past power heart future create john

------ Topic 9 ------
thomas unknown einstein albert act franklin go way desire edison



In [15]:
# Compute Perplexity
## a measure of how good the model is. lower the better
base_perplexity = base_model.log_perplexity(corpus)
print('\nPerplexity: ', base_perplexity) 

# Compute Coherence Score
coherence_model = CoherenceModel(model=base_model, texts=tweets_cleaned['unigram_tokens'], 
                                   dictionary=id2word, coherence='c_v')
coherence_lda_model_base = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_base)


Perplexity:  -7.208987546547177

Coherence Score:  0.44070351906470073


In [16]:
# topic distance visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(base_model, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.056503  0.088794       1        1  11.109446
4     -0.224757 -0.047185       2        1  10.549174
9      0.015787 -0.007646       3        1  10.506811
6     -0.119734  0.106982       4        1   9.883570
8      0.219477 -0.117423       5        1   9.829756
1     -0.114587  0.039100       6        1   9.827788
0      0.043047  0.150293       7        1   9.673932
2      0.130812  0.225106       8        1   9.636506
3     -0.138621 -0.284147       9        1   9.529160
7      0.245078 -0.153873      10        1   9.453858, topic_info=        Term           Freq          Total Category  logprob  loglift
2060    dyer  104056.000000  104056.000000  Default  30.0000  30.0000
1331       u   32374.000000   32374.000000  Default  29.0000  29.0000
94    change   26575.000000   26575.000000  Default  28.0000  28.0000
8       know   31258.000000   31258.000000  Default  27.0000  27.0000
224     like   23376.000000   23376.000000  Default  26.0000  26.0000
...      ...            ...            ...      ...      ...      ...
113    heart    7211.807675   19088.459773  Topic10  -4.0199   1.3854
46      good    7444.789974   29593.129205  Topic10  -3.9881   0.9787
116    thing    6440.351610   40500.315100  Topic10  -4.1330   0.5200
768      end    3336.311595    8383.123193  Topic10  -4.7907   1.4374
601    begin    3121.315753    5127.987600  Topic10  -4.8573   1.8623

[660 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
267       1  0.852567  ability
267       5  0.003574  ability
267       6  0.142094  ability
267       8  0.001702  ability
204       1  0.989147  abraham
...     ...       ...      ...
76        7  0.000308     year
76        9  0.021875     year
4851      3  0.999761     yogi
4863      3  0.001407      zen
4863     10  0.998410      zen

[1697 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 5, 10, 7, 9, 2, 1, 3, 4, 8])

In [17]:
get_document_topics = [base_model.get_document_topics(item) for item in corpus]

In [18]:
len(get_document_topics)

621449

In [19]:
get_document_topics[:20]

[[(0, 0.07348614),
  (1, 0.08435536),
  (2, 0.0638314),
  (3, 0.3541693),
  (5, 0.100295246),
  (6, 0.047012977),
  (8, 0.12144657),
  (9, 0.14540052)],
 [(0, 0.1517204),
  (3, 0.16693538),
  (4, 0.04887485),
  (5, 0.037234828),
  (6, 0.19258387),
  (7, 0.28993842),
  (8, 0.099666014)],
 [(0, 0.06419782),
  (1, 0.092184685),
  (3, 0.04897039),
  (4, 0.46670556),
  (6, 0.12483571),
  (7, 0.18571176)],
 [(3, 0.096881375),
  (4, 0.12258687),
  (5, 0.11568396),
  (6, 0.14210698),
  (8, 0.3732157),
  (9, 0.13212995)],
 [(1, 0.19586238),
  (4, 0.10088818),
  (5, 0.093007624),
  (6, 0.1885159),
  (8, 0.22462039),
  (9, 0.17891636)],
 [(1, 0.05394945),
  (3, 0.11788311),
  (4, 0.07583698),
  (5, 0.11860439),
  (6, 0.34843567),
  (8, 0.26862094)],
 [(1, 0.35369945),
  (4, 0.19746833),
  (6, 0.065758675),
  (8, 0.108351775),
  (9, 0.25090736)],
 [(0, 0.014285883),
  (1, 0.15474057),
  (2, 0.014288052),
  (3, 0.29936028),
  (4, 0.014286218),
  (5, 0.014285798),
  (6, 0.01428674),
  (7, 0.15962894

In [24]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           alpha=a,
                                           eta=b,
                                           random_state=222)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [25]:
# start preprocess runtime
start_time = time.time() 

# Topics range
min_topics = 11
max_topics = 18
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = [.01,.1,.5,1,'symmetric','asymmetric']
# Beta parameter
beta = [.01,.1,.5,1,'symmetric']

# model results dict
model_results = {'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=210)

    # iterate through number of topics
    for k in topics_range:
        # iterate through alpha values
        for a in alpha:
            # iterare through beta values
            for b in beta:
                # get the coherence score for the given parameters
                cv = compute_coherence_values(corpus=corpus, dictionary=id2word, 
                                              k=k, a=a, b=b)
                # Save the model results
                model_results['Topics'].append(k)
                model_results['Alpha'].append(a)
                model_results['Beta'].append(b)
                model_results['Coherence'].append(cv)
                
                pbar.update(1)
    lda_tuning_results = pd.DataFrame(model_results)                
    lda_tuning_results.to_csv('/content/drive/MyDrive/Data/NLP_Capstone/lda_tuning_no_ngrams_11-18.csv', index=False)
    pbar.close()

# print preprocess runtime
print(time.strftime(f'%H hours, %M minutes, %S seconds', time.gmtime(time.time() - start_time)))



  0%|          | 0/210 [00:00<?, ?it/s]

  0%|          | 1/210 [01:34<5:30:50, 94.98s/it]

  1%|          | 2/210 [03:12<5:32:18, 95.86s/it]

  1%|▏         | 3/210 [04:54<5:36:11, 97.45s/it]

  2%|▏         | 4/210 [06:35<5:38:11, 98.50s/it]

  2%|▏         | 5/210 [08:13<5:36:18, 98.43s/it]

  3%|▎         | 6/210 [09:43<5:26:38, 96.07s/it]

  3%|▎         | 7/210 [11:17<5:22:29, 95.32s/it]

  4%|▍         | 8/210 [12:52<5:20:44, 95.27s/it]

  4%|▍         | 9/210 [14:28<5:19:34, 95.40s/it]

  5%|▍         | 10/210 [16:02<5:16:31, 94.96s/it]

  5%|▌         | 11/210 [17:39<5:17:15, 95.66s/it]

  6%|▌         | 12/210 [19:21<5:22:10, 97.63s/it]

  6%|▌         | 13/210 [21:04<5:25:51, 99.25s/it]

  7%|▋         | 14/210 [22:47<5:27:43, 100.33s/it]

  7%|▋         | 15/210 [24:29<5:27:42, 100.83s/it]

  8%|▊         | 16/210 [25:58<5:14:26, 97.25s/it] 

  8%|▊         | 17/210 [27:29<5:07:11, 95.50s/it]

  9%|▊         | 18/210 [29:02<5:03:17, 94.78s/it]

  9%|▉         | 19/210 [30

05 hours, 53 minutes, 38 seconds


In [26]:
lda_tuning_results.sort_values('Coherence',ascending=False).head(30)

,Topics,Alpha,Beta,Coherence
132,15,0.5,0.5,0.538174
134,15,0.5,symmetric,0.534644
131,15,0.5,0.1,0.533995
104,14,0.5,symmetric,0.533336
101,14,0.5,0.1,0.532901
102,14,0.5,0.5,0.531125
190,17,0.5,0.01,0.529044
161,16,0.5,0.1,0.528460
206,17,asymmetric,0.1,0.527969
164,16,0.5,symmetric,0.527800


In [27]:
# number optimal of topics
num_topics = 8
# build optimal LDA model
optimal_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                        alpha=.5,
                                        eta=.1,
                                        random_state=222
                                        )
# filtering for words 
words = [re.findall(r'"([^"]*)"',t[1]) for t in optimal_model.print_topics()]

# create topic sorted by 10 most relevent words
topics = [' '.join(t[0:10]) for t in words]


# print most relevent words for each topic
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
time man choice right work proverb strong lose moment love

------ Topic 1 ------
love mind peace give dream strength happen place lie happy

------ Topic 2 ------
life amp way day person learn live find well matter

------ Topic 3 ------
good people want try self result action jim make far

------ Topic 4 ------
like look change fear small believe fail roosevelt henry important

------ Topic 5 ------
dyer u thing think act thought feel thomas man great

------ Topic 6 ------
world success man choose create einstein albert opportunity franklin churchill

------ Topic 7 ------
know come heart light get let lincoln abraham thing lead



In [28]:
# Compute Perplexity
## a measure of how good the model is. lower the better
base_perplexity = optimal_model.log_perplexity(corpus)
print('\nPerplexity: ', base_perplexity) 

# Compute Coherence Score
coherence_model = CoherenceModel(model=optimal_model, texts=tweets_cleaned['unigram_tokens'], 
                                   dictionary=id2word, coherence='c_v')
coherence_lda_model_base = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_base)


Perplexity:  -7.143465477742901

Coherence Score:  0.4493182478610913


In [29]:
# topic distance visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(optimal_model, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.230806  0.148309       1        1  13.305961
7     -0.192450 -0.127848       2        1  13.150700
5     -0.089951 -0.028959       3        1  12.840060
0     -0.129463  0.136802       4        1  12.499195
4     -0.081611 -0.205562       5        1  12.493027
1     -0.113066  0.190346       6        1  12.286586
3      0.238228 -0.232656       7        1  11.900739
6      0.137508  0.119569       8        1  11.523731, topic_info=             Term           Freq          Total Category  logprob  loglift
2060         dyer  113881.000000  113881.000000  Default  30.0000  30.0000
124          life   62689.000000   62689.000000  Default  29.0000  29.0000
188          time   37266.000000   37266.000000  Default  28.0000  28.0000
46           good   33600.000000   33600.000000  Default  27.0000  27.0000
93            amp   42285.000000   42285.000000  Default  26.0000  26.0000
...           ...            ...            ...      ...      ...      ...
379       respect    5667.091512    7828.698843   Topic8  -4.4589   1.8376
358           man   11247.856568   31846.709439   Topic8  -3.7734   1.1200
346       failure    4961.908665    7419.738263   Topic8  -4.5918   1.7584
954   achievement    3781.907507    4660.182997   Topic8  -4.8633   1.9519
510         power    4824.425046   14228.054130   Topic8  -4.6199   1.0792

[564 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
468       4  0.017584    able
468       5  0.011723    able
468       6  0.001332    able
468       7  0.968473    able
468       8  0.001066    able
...     ...       ...     ...
4851      2  0.995887    yogi
4851      3  0.004129    yogi
4863      7  0.999642     zen
240       2  0.999345     zig
241       2  0.999282  ziglar

[1872 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 8, 6, 1, 5, 2, 4, 7])

In [30]:
# number optimal of topics
num_topics = 7
# build optimal LDA model
optimal_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                        alpha=.5,
                                        eta=.1,
                                        random_state=222
                                        )
# filtering for words 
words = [re.findall(r'"([^"]*)"',t[1]) for t in optimal_model.print_topics()]

# create topic sorted by 10 most relevent words
topics = [' '.join(t[0:10]) for t in words]


# print most relevent words for each topic
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
time man get love choice right work proverb make lose

------ Topic 1 ------
heart love let mind peace give thomas strength dream happen

------ Topic 2 ------
life amp way day look learn person find change new

------ Topic 3 ------
good people want try self john result change action attract

------ Topic 4 ------
like fear end small believe great fall succeed fail roosevelt

------ Topic 5 ------
dyer u thing think thought act great man lincoln abraham

------ Topic 6 ------
know come world success albert einstein man create choose unknown



In [31]:
# Compute Perplexity
## a measure of how good the model is. lower the better
base_perplexity = optimal_model.log_perplexity(corpus)
print('\nPerplexity: ', base_perplexity) 

# Compute Coherence Score
coherence_model = CoherenceModel(model=optimal_model, texts=tweets_cleaned['unigram_tokens'], 
                                   dictionary=id2word, coherence='c_v')
coherence_lda_model_base = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_base)


Perplexity:  -7.156468327452237

Coherence Score:  0.41163016099824995


In [32]:
# topic distance visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(optimal_model, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.002634 -0.172780       1        1  15.137749
0     -0.187859  0.123171       2        1  14.931382
5     -0.027889  0.281311       3        1  14.630188
1     -0.208981 -0.030342       4        1  14.307246
3      0.230479 -0.067375       5        1  14.010685
6     -0.060178 -0.238366       6        1  13.538134
4      0.251794  0.104381       7        1  13.444617, topic_info=        Term           Freq          Total Category  logprob  loglift
2060    dyer  113703.000000  113703.000000  Default  30.0000  30.0000
124     life   63794.000000   63794.000000  Default  29.0000  29.0000
188     time   36365.000000   36365.000000  Default  28.0000  28.0000
46      good   33441.000000   33441.000000  Default  27.0000  27.0000
224     like   26212.000000   26212.000000  Default  26.0000  26.0000
...      ...            ...            ...      ...      ...      ...
160     step    5150.829186    9758.716130   Topic7  -4.7086   1.3676
94    change    6495.969063   24154.624466   Topic7  -4.4765   0.6933
440   inside    4217.999093    5958.189485   Topic7  -4.9084   1.6612
162    think    4766.659373   28988.568380   Topic7  -4.7861   0.2013
116    thing    3973.907798   34514.950986   Topic7  -4.9680  -0.1551

[527 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
267       1  0.102148  ability
267       2  0.853867  ability
267       3  0.000188  ability
267       4  0.007713  ability
267       5  0.001505  ability
...     ...       ...      ...
4851      2  0.000241     yogi
4851      3  0.023642     yogi
4851      6  0.976091     yogi
4863      5  0.999687      zen
241       7  0.999030   ziglar

[1614 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 6, 2, 4, 7, 5])